In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.datasets import make_classification
from typing import Dict, List, Tuple, Any
import matplotlib.patches as mpatches
import kagglehub
import glob
import os

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

- Tạo class DoSDDoSClassifier với đầy đủ phương pháp:
- Triển khai 4 thuật toán ML (KNN, AdaBoost, Random Forest, SVM)
- Triển khai 3 kỹ thuật giảm chiều dữ liệu (PCA, Feature Importance, Univariate Selection)
- Tiền xử lý dữ liệu (làm sạch, chuẩn hóa MinMax)

In [2]:
class DoSDDoSClassifier:
    """
    - Thuật toán 4 ml: KNN, AdaBoost, Random Forest, SVM
    - 3 Kỹ thuật giảm chiều dữ liệu: PCA, Feature Importance, Univariate Selection
    - Hoàn thành tiền xử lý
    - Đánh giá với thời gian và độ chính xác
    """
    
    def __init__(self):
        # Thuật toán ML
        self.ml_algorithms = {
            'KNN': KNeighborsClassifier(n_neighbors=5),
            'AdaBoost': AdaBoostClassifier(n_estimators=50, random_state=42),
            'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
            'SVM': SVC(kernel='rbf', random_state=42)
        }
        
        # Giảm chiều dữ liệu
        self.feature_selection_methods = {
            'PCA': None,  # Sẽ được khởi tạo động
            'Feature_Importance': None,  # ExtraTree + SelectFromModel
            'Univariate_Selection': None  # SelectKBest với chi2
        }
        
        # Scaler normalisation MinMax (-1, 1)
        self.scaler = MinMaxScaler(feature_range=(-1, 1))
        self.label_encoder = LabelEncoder()
        
        # Lưu trữ kết quả và dữ liệu đã giảm chiều
        self.results = {}
        self.feature_reduced_data = {}
        
    def preprocess_data(self, X, y, dataset_name="Dataset"):
        """
        Tiền xử lý dữ liệu:
        1. Làm sạch giá trị NaN và vô cùng
        2. Loại bỏ các features bằng 0
        3. Chuẩn hóa MinMax (-1, 1)
        4. Encode nhãn
        """
        print(f"\nTiền xử lý dữ liệu {dataset_name}...")
        print(f"   Kích thước ban đầu: {X.shape}")
        
        # 1. Làm sạch dữ liệu
        X_clean = X.copy()
        X_clean = X_clean.replace([np.inf, -np.inf], np.nan)
        X_clean = X_clean.fillna(X_clean.mean())

        # 2. Loại bỏ các feature bằng 0
        non_zero_var_cols = X_clean.columns[(X_clean != 0).any(axis=0)]
        X_clean = X_clean[non_zero_var_cols]
        
        print(f"   Sau khi làm sạch: {X_clean.shape}")
        
        # 3. Chuẩn hóa MinMax (-1, 1)
        X_scaled = pd.DataFrame(
            self.scaler.fit_transform(X_clean),
            columns=X_clean.columns,
            index=X_clean.index
        )
        
        # 4. Encode nhãn
        y_encoded = self.label_encoder.fit_transform(y)
        
        print(f"Tiền xử lý hoàn tất - shape cuối cùng: {X_scaled.shape}")
        return X_scaled, y_encoded
    
    def apply_feature_selection(self, X_train, X_test, y_train, target_features=None):
        """
        Áp dụng 3 kỹ thuật giảm chiều dữ liệu:
        1. PCA (giảm xuống ~50% số feature)
        2. Feature Importance (ExtraTree + SelectFromModel)
        3. Univariate Selection (SelectKBest với chi2)
        """
        print(f"\nÁp dụng các kỹ thuật giảm chiều dữ liệu...")
        
        # Tính số lượng features mục tiêu (khoảng 50% theo bài báo)
        if target_features is None:
            target_features = max(X_train.shape[1] // 2, 10)
        
        results = {}
        
        # 1. PCA (giảm xuống ~50% số features)
        print(f"PCA: {X_train.shape[1]} → {target_features} features")
        pca = PCA(n_components=target_features)
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)
        results['PCA'] = {
            'X_train': X_train_pca,
            'X_test': X_test_pca,
            'n_features': target_features,
            'method': pca
        }
        
        # 2. Feature Importance ExtraTree + SelectFromModel
        print(f"Feature Importance: tự động chọn các features tốt nhất")
        extra_tree = ExtraTreesClassifier(n_estimators=100, random_state=42)
        extra_tree.fit(X_train, y_train)
        selector = SelectFromModel(extra_tree, max_features=target_features)
        X_train_fi = selector.fit_transform(X_train, y_train)
        X_test_fi = selector.transform(X_test)
        results['Feature_Importance'] = {
            'X_train': X_train_fi,
            'X_test': X_test_fi,
            'n_features': X_train_fi.shape[1],
            'method': selector
        }

        # 3. Univariate Selection với chi2 (bài báo trang 3)
        print(f"Univariate Selection: SelectKBest vs chi2")
        # Chuyển đổi thành giá trị dương cho chi2
        X_train_pos = X_train - X_train.min() + 0.01
        X_test_pos = X_test - X_test.min() + 0.01
        
        univariate = SelectKBest(score_func=chi2, k=target_features)
        X_train_us = univariate.fit_transform(X_train_pos, y_train)
        X_test_us = univariate.transform(X_test_pos)
        results['Univariate_Selection'] = {
            'X_train': X_train_us,
            'X_test': X_test_us,
            'n_features': target_features,
            'method': univariate
        }
        
        print(f"Hoàn thành giảm chiều dữ liệu")
        return results
    
    def evaluate_model_combination(self, X_train, X_test, y_train, y_test, 
                                 ml_name, feature_method_name, dataset_name):
        """
        Đánh giá thời gian và độ chính xác
        """
        # Lấy mô hình ML
        model = self.ml_algorithms[ml_name]
        
        # Huấn luyện vs đo thời gian
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time
        
        # Dự đoán vs đo thời gian
        start_time = time.time()
        y_pred = model.predict(X_test)
        predict_time = time.time() - start_time

        # Tính độ chính xác
        accuracy = accuracy_score(y_test, y_pred)

        # Tổng thời gian (huấn luyện + dự đoán) tính bằng mili giây
        total_time_ms = (train_time + predict_time) * 1000
        
        return {
            'accuracy': accuracy * 100,  # Phần trăm
            'total_time_ms': total_time_ms,
            'train_time': train_time,
            'predict_time': predict_time,
            'y_pred': y_pred
        }


- Tạo các bộ dữ liệu tổng hợp mô phỏng NSL-KDD 2019, CICIDS 2017 và dữ liệu mô phỏng (đoạn này bỏ qua vì dùng luôn 2 bộ dữ liệu trên kaggle)

In [3]:
#bỏ qua
def create_synthetic_datasets():
    """
    Tạo các bộ dữ liệu tổng hợp mô phỏng đặc trưng của 3 bộ dữ liệu trong bài báo:
    1. NSL-KDD 2019: ~150,000 mẫu, 42 đặc trưng → 21 sau giảm
    2. CICIDS 2017: 68 đặc trưng → 23 sau giảm  
    3. Dữ liệu mô phỏng: 45,500 mẫu, 73 đặc trưng → 20 sau giảm
    """
    
    datasets = {}
    
    print("Tạo các bộ dữ liệu tổng hợp dựa trên bài báo...")
    
    # 1. Dataset simulant NSL-KDD 2019
    print("\nTạo bộ dữ liệu NSL-KDD 2019...")
    X_nsl, y_nsl = make_classification(
        n_samples=15000,  # số lượng mẫu (article: 150,000)
        n_features=42,    
        n_informative=35,
        n_redundant=5,
        n_clusters_per_class=2,
        class_sep=0.8,
        random_state=42
    )
    
    # Chuyển đổi sang dữ liệu với tên cột thực tế
    nsl_features = [f'feature_{i+1}' for i in range(42)]
    X_nsl_df = pd.DataFrame(X_nsl, columns=nsl_features)
    y_nsl_labels = ['Normal' if label == 0 else 'Attack' for label in y_nsl]
    
    datasets['NSL-KDD_2019'] = {
        'X': X_nsl_df,
        'y': pd.Series(y_nsl_labels),
        'description': 'Dataset tổng hợp mô phỏng NSL-KDD 2019 (42→21 đặc trưng)'
    }
    
    # 2. Dataset simulant CICIDS 2017
    print("Tạo bộ dữ liệu tổng hợp CICIDS 2017...")
    X_cicids, y_cicids = make_classification(
        n_samples=12000,  
        n_features=68,    
        n_informative=55,
        n_redundant=8,
        n_clusters_per_class=3,
        class_sep=0.7,
        random_state=123
    )

    # Mô phỏng các loại tấn công khác nhau
    cicids_features = [f'flow_feature_{i+1}' for i in range(68)]
    X_cicids_df = pd.DataFrame(X_cicids, columns=cicids_features)

    # Tạo nhãn đa lớp để mô phỏng các loại tấn công khác nhau
    attack_types = ['Normal', 'DoS', 'DDoS', 'Brute_Force']
    y_cicids_multiclass = np.random.choice(attack_types, size=len(y_cicids), p=[0.6, 0.2, 0.15, 0.05])
    
    datasets['CICIDS_2017'] = {
        'X': X_cicids_df,
        'y': pd.Series(y_cicids_multiclass),
        'description': 'Dataset tổng hợp mô phỏng CICIDS 2017 (68→23 đặc trưng)'
    }

    # 3. Dataset mô phỏng dữ liệu (Lima Filho et al., 2019)
    print("Tạo bộ dữ liệu mô phỏng...")
    X_sim, y_sim = make_classification(
        n_samples=4550,   
        n_features=73,   
        n_informative=60,
        n_redundant=10,
        n_clusters_per_class=2,
        class_sep=0.9,    # Rất tách biệt để mô phỏng một môi trường kiểm soát
        random_state=456
    )
    
    sim_features = [f'network_metric_{i+1}' for i in range(73)]
    X_sim_df = pd.DataFrame(X_sim, columns=sim_features)
    y_sim_labels = ['Normal' if label == 0 else 'Attack' for label in y_sim]
    
    datasets['Simulated_Data'] = {
        'X': X_sim_df,
        'y': pd.Series(y_sim_labels),
        'description': 'Dataset tổng hợp mô phỏng dữ liệu Lima Filho (73→20 đặc trưng)'
    }
    
    return datasets

# Tạo các bộ dữ liệu tổng hợp
synthetic_datasets = create_synthetic_datasets()

Tạo các bộ dữ liệu tổng hợp dựa trên bài báo...

Tạo bộ dữ liệu NSL-KDD 2019...
Tạo bộ dữ liệu tổng hợp CICIDS 2017...
Tạo bộ dữ liệu mô phỏng...


In [4]:
# Bổ sung một vài giá trị có vấn đề để kiểm tra tiền xử lý
print("Bổ sung giá trị NaN và vô cực để kiểm tra tiền xử lý...")

for dataset_name, dataset in synthetic_datasets.items():
    # Tiêm 1-2% giá trị có vấn đề
    n_samples, n_features = dataset['X'].shape
    n_nan = int(0.01 * n_samples * n_features)

    # Vị trí ngẫu nhiên cho NaN
    nan_positions = np.random.choice(n_samples * n_features, n_nan, replace=False)
    for pos in nan_positions:
        row, col = divmod(pos, n_features)
        dataset['X'].iloc[row, col] = np.nan

    # Một vài giá trị vô cực
    n_inf = max(1, n_nan // 10)
    inf_positions = np.random.choice(n_samples * n_features, n_inf, replace=False)
    for pos in inf_positions:
        row, col = divmod(pos, n_features)
        dataset['X'].iloc[row, col] = np.inf if np.random.random() > 0.5 else -np.inf

print(f"\n✅ 3 datasets tổng hợp được tạo thành công:")
for name, dataset in synthetic_datasets.items():
    print(f"   • {name}: {dataset['X'].shape[0]} mẫu, {dataset['X'].shape[1]} đặc trưng")
    print(f"     Classes: {dataset['y'].value_counts().to_dict()}")
    print(f"     NaN values: {dataset['X'].isnull().sum().sum()}")
    print(f"     Inf values: {np.isinf(dataset['X']).sum().sum()}")

Bổ sung giá trị NaN và vô cực để kiểm tra tiền xử lý...

✅ 3 datasets tổng hợp được tạo thành công:
   • NSL-KDD_2019: 15000 mẫu, 42 đặc trưng
     Classes: {'Normal': 7500, 'Attack': 7500}
     NaN values: 6296
     Inf values: 630
   • CICIDS_2017: 12000 mẫu, 68 đặc trưng
     Classes: {'Normal': 7233, 'DoS': 2376, 'DDoS': 1791, 'Brute_Force': 600}
     NaN values: 8150
     Inf values: 816
   • Simulated_Data: 4550 mẫu, 73 đặc trưng
     Classes: {'Attack': 2279, 'Normal': 2271}
     NaN values: 3319
     Inf values: 332


- Load data từ bộ dataset NSL-KDD 2019, CICIDS 2017 trên kaggle
- Với bộ NSL-KDD 2019: gán tên cột, encode một số cột dữ liệu, gộp 2 bộ dữ liệu train, test
- Với CICIDS 2017: gộp tất cả các bộ dữ liệu

In [5]:
def load_dataset():
  
  datasets = {}
  # Lấy dữ liệu bộ NSL-KDD 2019
  # Download latest version
  path_nslkdd = kagglehub.dataset_download("hassan06/nslkdd")
  train_data_KDD = pd.read_csv(f"{path_nslkdd}/KDDTrain+.txt", header=None)
  test_data_KDD = pd.read_csv(f"{path_nslkdd}/KDDTest+.txt", header=None)
  
  # Define the list of column names based on the NSL-KDD dataset description
  columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
    'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
    'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds',
    'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
    'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
    'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'attack', 'level'
  ]

  # Assign the column names to the dataframe
  train_data_KDD.columns = columns
  test_data_KDD.columns = columns
  
  data_KDD = pd.concat([train_data_KDD, test_data_KDD], ignore_index=True)

  X_KDD = data_KDD.drop(columns=['attack', 'level'])
  for col in X_KDD.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_KDD[col] = le.fit_transform(X_KDD[col])
  y_KDD = data_KDD['attack']

  datasets['NSL-KDD'] = {
        'X': X_KDD,
        'y': y_KDD,
        'description': 'NSL-KDD dataset từ Kaggle'
    }

  # Lấy dữ liệu bộ CICIDS 2017
  # Download latest version
  path_cicids = kagglehub.dataset_download("chethuhn/network-intrusion-dataset")

  # Get all CSV files in the directory
  csv_files = glob.glob(os.path.join(path_cicids, "*.csv"))
  # Read and concatenate all CSV files into a single DataFrame
  df_2017 = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)
  df_2017.columns = df_2017.columns.str.strip()
  #print(df_2017['Label'].value_counts())
  df_2017['Label'] = df_2017['Label'].str.strip()

  X_cicids = df_2017.drop(columns=['Label'])
  y_cicids = df_2017['Label']
  datasets['CICIDS_2017'] = {
        'X': X_cicids,
        'y': y_cicids,
        'description': 'CICIDS 2017 dataset từ Kaggle'
    }
  return datasets
datasets = load_dataset()    

In [6]:
print((datasets))

{'NSL-KDD': {'X':         duration  protocol_type  service  flag  src_bytes  dst_bytes  land  \
0              0              1       20     9        491          0     0   
1              0              2       44     9        146          0     0   
2              0              1       49     5          0          0     0   
3              0              1       24     9        232       8153     0   
4              0              1       24     9        199        420     0   
...          ...            ...      ...   ...        ...        ...   ...   
148512         0              1       54     9        794        333     0   
148513         0              1       24     9        317        938     0   
148514         0              1       24     9      54540       8314     0   
148515         0              2       12     9         42         42     0   
148516         0              1       57     1          0          0     0   

        wrong_fragment  urgent  hot  ...  dst

Implement systematic evaluation with time/accuracy measurement 
- Triển khai đánh giá hệ thống vá đo thời gian/độ chính xác

In [6]:
def run_complete_evaluation(classifier, datasets):
    """
    Thực hiện đánh giá đầy đủ theo phương pháp của bài viết:
    - Kiểm tra các thuật toán 4 ml với 3 kỹ thuật lựa chọn tính năng
    - Đo lường thời gian thực hiện và độ chính xác
    - Tạo bảng kết quả
    """
    
    print("Bắt đầu đánh giá")
    print("="*80)
    
    all_results = {}
    

    for dataset_name, dataset_info in datasets.items():
        print(f"\nĐánh giá bộ dữ liệu: {dataset_name}")
        print(f"   {dataset_info['description']}")
        print("-" * 60)
        
        X, y = dataset_info['X'], dataset_info['y']
        
        # 1. Tiền xử lý
        X_processed, y_processed = classifier.preprocess_data(X, y, dataset_name)

        # 2. Division train/test (80/20)
        X_train, X_test, y_train, y_test = train_test_split(
            X_processed, y_processed, test_size=0.2, random_state=42, stratify=y_processed
        )
        
        print(f"\n   Division:")
        print(f"      Train: {X_train.shape[0]} mẫu")
        print(f"      Test: {X_test.shape[0]} mẫu")
        
        # 3. Áp dụng các kỹ thuật lựa giảm chiều
        feature_selection_results = classifier.apply_feature_selection(
            X_train, X_test, y_train
        )
        
        # 4. Đánh giá kết hợp ML + Feature Selection
        dataset_results = {}
        
        print(f"\n   ĐÁNH GIÁ CÁC KẾT HỢP ML + Feature Selection:")
        print(f"   {'Thuật toán':<15} {'Phương pháp giảm chiều':<18} {'Độ chính xác (%)':<12} {'Thời gian (ms)':<10} {'Số features':<8}")
        print(f"   {'-'*15} {'-'*18} {'-'*12} {'-'*10} {'-'*8}")
        
        # Đánh giá trước khi giảm chiều (baseline)
        print(f"\n   BASELINE (không giảm chiều):")
        for ml_name in classifier.ml_algorithms.keys():
            result = classifier.evaluate_model_combination(
                X_train, X_test, y_train, y_test, ml_name, "No_Reduction", dataset_name
            )
            
            dataset_results[f"{ml_name}_No_Reduction"] = {
                'ml_algorithm': ml_name,
                'feature_method': 'No_Reduction',
                'accuracy': result['accuracy'],
                'time_ms': result['total_time_ms'],
                'n_features': X_train.shape[1]
            }
            
            print(f"   {ml_name:<15} {'No_Reduction':<18} {result['accuracy']:<12.2f} {result['total_time_ms']:<10.1f} {X_train.shape[1]:<8}")
        
        # đánh giá khi giảm chiều
        print(f"\n   SAU KHI GIẢM CHIỀU:")
        for feature_method, feature_data in feature_selection_results.items():
            print(f"\n   🔹 {feature_method}:")
            
            X_train_reduced = feature_data['X_train']
            X_test_reduced = feature_data['X_test']
            n_features = feature_data['n_features']
            
            for ml_name in classifier.ml_algorithms.keys():
                result = classifier.evaluate_model_combination(
                    X_train_reduced, X_test_reduced, y_train, y_test, 
                    ml_name, feature_method, dataset_name
                )
                
                key = f"{ml_name}_{feature_method}"
                dataset_results[key] = {
                    'ml_algorithm': ml_name,
                    'feature_method': feature_method,
                    'accuracy': result['accuracy'],
                    'time_ms': result['total_time_ms'],
                    'n_features': n_features
                }
                
                print(f"     {ml_name:<15} {feature_method:<18} {result['accuracy']:<12.2f} {result['total_time_ms']:<10.1f} {n_features:<8}")
        
        all_results[dataset_name] = dataset_results
        
        # Xác định sự kết hợp tốt nhất cho bộ dữ liệu này
        best_combo = max(dataset_results.items(), key=lambda x: x[1]['accuracy'])
        print(f"\n   Sự kết hợp tốt nhất {dataset_name}:")
        print(f"      {best_combo[1]['ml_algorithm']} + {best_combo[1]['feature_method']}")
        print(f"      Độ chính xác: {best_combo[1]['accuracy']:.2f}%")
        print(f"      Thời gian: {best_combo[1]['time_ms']:.1f} ms")
        print(f"      Số đặc điểm: {best_combo[1]['n_features']}")

    return all_results

# Khởi tạo trình phân loại và thực hiện đánh giá
classifier = DoSDDoSClassifier()
#evaluation_results = run_complete_evaluation(classifier, load_dataset())
evaluation_results = run_complete_evaluation(classifier, create_synthetic_datasets())


Tạo các bộ dữ liệu tổng hợp dựa trên bài báo...

Tạo bộ dữ liệu NSL-KDD 2019...
Tạo bộ dữ liệu tổng hợp CICIDS 2017...
Tạo bộ dữ liệu mô phỏng...
Bắt đầu đánh giá

Đánh giá bộ dữ liệu: NSL-KDD_2019
   Dataset tổng hợp mô phỏng NSL-KDD 2019 (42→21 đặc trưng)
------------------------------------------------------------

Tiền xử lý dữ liệu NSL-KDD_2019...
   Kích thước ban đầu: (15000, 42)
   Sau khi làm sạch: (15000, 42)
Tiền xử lý hoàn tất - shape cuối cùng: (15000, 42)

   Division:
      Train: 12000 mẫu
      Test: 3000 mẫu

Áp dụng các kỹ thuật giảm chiều dữ liệu...
PCA: 42 → 21 features
Feature Importance: tự động chọn các features tốt nhất
Univariate Selection: SelectKBest vs chi2
Hoàn thành giảm chiều dữ liệu

   ĐÁNH GIÁ CÁC KẾT HỢP ML + Feature Selection:
   Thuật toán      Phương pháp giảm chiều Độ chính xác (%) Thời gian (ms) Số features
   --------------- ------------------ ------------ ---------- --------

   BASELINE (không giảm chiều):
   KNN             No_Reduction     

- Tạo bảng kết quả

In [7]:
def create_results_tables(evaluation_results):
    """
    tạo bảng kết quả
    """
    
    print("TẠO CÁC BẢNG KẾT QUẢ")
    print("="*80)
    
    for dataset_name, results in evaluation_results.items():
        print(f"\nBẢNG KẾT QUẢ - {dataset_name}")
        print("-" * 70)

        # tạo 1 DataFrame để hiển thị bảng
        table_data = []
        
        for combo_name, combo_results in results.items():
            ml_algo = combo_results['ml_algorithm']
            feature_method = combo_results['feature_method']
            accuracy = combo_results['accuracy']
            time_ms = combo_results['time_ms']
            n_features = combo_results['n_features']
            
            table_data.append({
                'Thuật toán ML': ml_algo,
                'Phương pháp giảm chiều': feature_method,
                'Độ chính xác (%)': f"{accuracy:.2f}",
                'Thời gian (ms)': f"{time_ms:.1f}",
                'Số features': n_features
            })
        
        df_results = pd.DataFrame(table_data)
        
        # Sắp xếp bằng cách giảm độ chính xác
        df_results = df_results.sort_values('Độ chính xác (%)', ascending=False)

        print(df_results.to_string(index=False))

        # Thống kê bổ sung
        best_accuracy = df_results.iloc[0]
        print(f"\n  KẾT QUẢ TỐT NHẤT:")
        print(f"      {best_accuracy['Thuật toán ML']} + {best_accuracy['Phương pháp giảm chiều']}")
        print(f"      Độ chính xác: {best_accuracy['Độ chính xác (%)']}%")
        print(f"      Thời gian: {best_accuracy['Thời gian (ms)']} ms")
        print(f"      Số features: {best_accuracy['Số features']}")

        # Phân tích cải thiện thời gian so với độ chính xác
        baseline_results = df_results[df_results['Phương pháp giảm chiều'] == 'No_Reduction']
        reduced_results = df_results[df_results['Phương pháp giảm chiều'] != 'No_Reduction']

        if not baseline_results.empty and not reduced_results.empty:
            avg_baseline_time = baseline_results['Thời gian (ms)'].str.replace(' ms', '').astype(float).mean()
            avg_reduced_time = reduced_results['Thời gian (ms)'].str.replace(' ms', '').astype(float).mean()

            time_improvement = ((avg_baseline_time - avg_reduced_time) / avg_baseline_time) * 100

            print(f"\n  PHÂN TÍCH CẢI THIỆN:")
            print(f"      Giảm thời gian trung bình: {time_improvement:.1f}%")
            print(f"      Thời gian trung bình baseline: {avg_baseline_time:.1f} ms")
            print(f"      Thời gian trung bình với giảm chiều: {avg_reduced_time:.1f} ms")

# Tạo bảng kết quả
create_results_tables(evaluation_results)

TẠO CÁC BẢNG KẾT QUẢ

BẢNG KẾT QUẢ - NSL-KDD_2019
----------------------------------------------------------------------
Thuật toán ML Phương pháp giảm chiều Độ chính xác (%) Thời gian (ms)  Số features
          SVM           No_Reduction            98.57         1098.3           42
          KNN           No_Reduction            97.00          335.4           42
 RandomForest           No_Reduction            93.60         4682.3           42
          SVM                    PCA            93.03         1396.5           21
          SVM   Univariate_Selection            90.70         1265.9           21
          SVM     Feature_Importance            90.40         1450.7           15
          KNN                    PCA            89.40           33.0           21
          KNN   Univariate_Selection            88.80           79.5           21
 RandomForest                    PCA            87.47         3285.2           21
          KNN     Feature_Importance            86.93      

Create performance visualizations and comparisons
- Tạo biểu đồ so sánh hiệu năng